In [ ]:
import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torch.utils.data as utils
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os
import platform
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -r DLAV-2022
!git clone https://github.com/vita-epfl/DLAV-2022.git
path = os.getcwd() + '/DLAV-2022/homeworks/hw2/test_batch'

rm: cannot remove 'DLAV-2022': No such file or directory
Cloning into 'DLAV-2022'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 83 (delta 31), reused 60 (delta 16), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [ ]:
# Set the variable to the location of the trained model
model_path = 'drive/MyDrive/Colab Notebooks/Cifar10_Classifier_pytorch_CUDA.ckpt'

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=3, n_output=10):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        # use the LeNet model
        # '?' denotes the batch size, which is 64 in our default setting from func above
        self.conv1 = nn.Conv2d(3, 6, 5)       # (?, 3,32,32) => (?, 6,28,28)
        self.pool = nn.MaxPool2d(2, 2)        # we use this pooling twice
                                              # 1st time :(?, 6,28,28) => (?, 6,14,14)
                                              # 2nd time :(?,16,10,10) => (?,16, 5, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)      # (?, 6,14,14) => (?,16,10,10)
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # (?,16, 5, 5) ~ (?,400) => (?,120)
        self.fc2 = nn.Linear(120, 84)         # (?,120) => (?,84)
        self.fc3 = nn.Linear(84, 10)          # (?, 84) => (?,10)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Tensor `x` with `x.view`.                                                    #
        ################################################################################
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # only keep the batch dim, flatten rest dims
        # alternatively 
        # x = x.view(x.size[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [ ]:
def predict_usingCNN(X):
    #########################################################################
    # TODO:                                                                 #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array                                 #
    # Note: For the predictions, you have to return the index of the max    #
    # value                                                                 #
    #########################################################################
    # CUDA setting
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = ConvNet().to(device)
    # Load my optimized model
    checkpoint = torch.load(model_path)
    net.load_state_dict(checkpoint)
    # prediction/inference
    with torch.no_grad():
        X = X.to(device)
        outputs = net(X)
        _, predicted = torch.max(outputs.data, 1)
        # _, predicted = torch.max(F.softmax(outputs,dim=1).data, 1)
        # cannot turn a CUDA tensor to numpy, so move to CPU first
        y_pred = predicted.to('cpu').numpy()
    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred
   

In [ ]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

In [ ]:
# Data Manipulation
mean_pytorch = np.array([0.4914, 0.4822, 0.4465])
std_pytorch = np.array([0.2023, 0.1994, 0.2010])
X_pytorch = np.divide(np.subtract( X/255 , mean_pytorch[np.newaxis, :,np.newaxis,np.newaxis]), std_pytorch[np.newaxis, :,np.newaxis,np.newaxis])

# Run Prediction and Evaluation
prediction_cnn = predict_usingCNN(torch.from_numpy(X_pytorch).float())
acc_cnn = sum(prediction_cnn == Y)/len(X_pytorch)
print("CNN Accuracy= %f"%(acc_cnn))

CNN Accuracy= 0.773300
